# Usage Example for the Laser Cross Calibration



In [ ]:
import laser_cross_calibration as lcc
from math import sin, cos, pi as PI, radians

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress

## Gunady Setup

### Setup
Gunady et al. used a 3D printer xyz-stage and two laser pointer for laser cross calibration. We can use their setup as an example and to validate the simulation pipeline.

![](./images/gunady_setup.png)
Source: [**DOI**: 10.1088/1361-6501/ad574d](https://iopscience.iop.org/article/10.1088/1361-6501/ad574d)

### Logic
The simulation logic works as follows:
1. Create a container for all surface components which can interact with the beams. 
    - The corresponding object is `OpticalSystem`
    - Create surface geometries which define the locations of optical interfaces in space
    - Define optical interfaces from a geometrie and two materials (pre and post surface)
    - Add the interface to the optical system
2. Define a laser beam source
    - In this case we use a `DualLaserStageSource` defining two laser beam sources with fixed geometrical relations
3. Create a tracer (`RayTracer`)
    - The tracer takes the system and a list of source to trace the rays emitted by the sources through the optical system.
4. Optionally the scene can be visualized using a `Scene` object.
    - Add all components to visualize and use the `make_figure` method to create a `plotly` graph


### Sketch
![](./images/gunady_setup_sketch.png)

The angles are `alpha = 11.5 deg` and `beta = 12.6 deg` according to Gunady et al.

### Logic Pipeline

In [ ]:
import mermaid as md
from mermaid.graph import Graph

render = md.Mermaid("""
graph LR
    Source(("Source<br/>(Ray Origin)"))
    Air["Air<br/>(Ray Propagation)"]
    FrontSurface{"Front Surface<br/>(Air-Glass Interface)<br/>Refraction"}
    Glass["Glass<br/>(Ray Propagation)"]
    BackSurface{"Back Surface<br/>(Glass-Water Interface)<br/>Refraction"}
    Water["Water<br/>(Ray Propagation)"]
    Intersection(("Intersection"))
                    
                    
    Source -->|"Ray"| Air
    Air -->|"Ray"| FrontSurface
    FrontSurface -->|"Ray"| Glass
    Glass -->|"Ray"| BackSurface
    BackSurface -->|"Ray"| Water     
    Water --> Intersection   
                    
    style Source fill:#008cff
    style Intersection fill:#1e8604
    style FrontSurface fill: #ffaa00
    style BackSurface fill: #ffaa00
""")

c = "#1e8604"
render

### Create Optical System

In [ ]:
# first a optical system instance is created. This will store all elements
system = lcc.tracing.OpticalSystem(final_propagation_distance=10)

# define the front of the glass plate at (0,0,0) with normal -y (outwards of the material!)
plane_front_surface = lcc.surfaces.Plane(
    point=lcc.constants.UNIT_Y_VECTOR3 * 0, normal=-lcc.constants.UNIT_Y_VECTOR3
)
# define the interface: geometry + materials on both sides
plane_front_interface = lcc.tracing.OpticalInterface(
    geometry=plane_front_surface,
    material_pre=lcc.materials.AIR,
    material_post=lcc.materials.GLASS_FUSED_SILICA,
)
# add to optical system
system.add_interface(plane_front_interface)

# define the back of the glass plate at (0,0.01,0), thickness 1cm, with normal +y (outwards of the material!)
# !!!NOTE THAT THE NORMAL DIRECTION HAS CHANGED!!!!
plane_back_surface = lcc.surfaces.Plane(
    point=lcc.constants.UNIT_Y_VECTOR3 * 1e-2, normal=lcc.constants.UNIT_Y_VECTOR3
)
plane_back_interface = lcc.tracing.OpticalInterface(
    geometry=plane_back_surface,
    material_pre=lcc.materials.GLASS_FUSED_SILICA,
    material_post=lcc.materials.WATER,
)
system.add_interface(plane_back_interface)

### Visualize the Optical system

In [ ]:
# Create a scene
scene = lcc.visualization.Scene()
scene.bounds_max = (0.5, 0.5, 0.5)
scene.bounds_min = (-0.5, -0.5, -0.5)
# Add the system
scene.add_system(system)
# Make a figure
scene.make_figure(
    display_bounds_max=(0.1, 0.1, 0.1), display_bounds_min=(-0.1, -0.1, -0.1)
)

### Create the source

In [ ]:
# the stage will be positioned at (0, -0.3, 0)
position = lcc.constants.ORIGIN_POINT3 - lcc.constants.UNIT_Y_VECTOR3 * 0.3

# the sources are spaced 20 cm apart
distance_between_sources = 0.2

# the angles defining the direction of the beams, according to the publication of Gunady et al
alpha = radians(11.5)
beta = radians(12.6)

direction_beam1 = np.array((-sin(alpha), cos(alpha), 0.0))
direction_beam2 = np.array((sin(beta), cos(beta), 0.0))

# build the actual source
source = lcc.sources.DualLaserStageSource(
    origin=position,
    arm1=lcc.constants.UNIT_X_VECTOR3 * distance_between_sources / 2,
    arm2=-lcc.constants.UNIT_X_VECTOR3 * distance_between_sources / 2,
    direction1=direction_beam1,
    direction2=direction_beam2,
)

### Visualize system and source

In [ ]:
# add the source to our scene
scene.clear_sources()
scene.add_source(source)
# make a figure, sources are displayed as colored cones, pointing in the direction of the emitted ray
scene.make_figure()

### Trace the system and find the intersection of rays

In [ ]:
# create a ray tracer object and add the defined system
tracer = lcc.tracing.RayTracer()
tracer.add_optical_system(system=system)

# trace the system using the defined source (supports multiple sources)
rays, intersection = tracer.trace_and_find_crossings(sources=[source])

scene.clear_rays()
scene.add_rays(rays=rays)
if intersection:
    x, y, z = intersection[0]
    scene.clear_points()
    scene.add_point(x=x, y=y, z=z)

scene.make_figure()

### Validate system
For their simple setup Gunady et al. specify the relation between y shifts of the stage and y shifts of the intersection point with **1.34**. To compare the simulation with their result the stage is moved stepwise in the y direction and the point of intersection is calculated. 


#### Simulate y shifts

In [ ]:
# copy the original position of the source
original_position = source.origin.copy()

# create some storages
intersections = []
rays_list = []
stage_shifts = []

# define a shift in y direction for the stage
total_shift = 0.25
number_of_steps = 10

step_dy = total_shift / number_of_steps

# shift the stage and calculate intersections
for i in range(number_of_steps):
    # calculate the intersection and store the results
    rays, intersection = tracer.trace_and_find_crossings(sources=[source])
    stage_shifts.append(i * step_dy)
    rays_list.append(rays)
    intersections.append(intersection[0])
    # move the stage by dy
    source.translate(y=step_dy)

# reset the position of the stage, needed for multiple runs of this cell
source.origin = original_position

#### Visualize y shifts

In [ ]:
fig, ax = plt.subplots()

ax.set_xlabel("y coordinate")
ax.set_ylabel("x coordinate")

ax.set_xlim(-0.35, 0.55)
ax.set_ylim(-0.2, 0.2)

ax.axvspan(
    plane_front_surface.point[1],
    plane_back_surface.point[1],
    alpha=0.8,
    facecolor="gray",
    edgecolor="black",
)

colors1 = plt.cm.Reds(np.linspace(0.2, 0.8, len(rays_list)))
colors2 = plt.cm.Blues(np.linspace(0.2, 0.8, len(rays_list)))

for rays, intersection, color1, color2 in zip(
    rays_list, intersections, colors1, colors2
):
    for ray, color in zip(rays, (color1, color2)):
        x, y, z = list(zip(*ray.path_positions, strict=True))
        ax.plot(y, x, c=color, marker="o", markersize=2.5, markeredgecolor="black")

    # mark stage y positions
    ax.axvline(rays[-1].path_positions[0][1], ls="--", c="orange", alpha=0.5)

    # mark intersection point
    ax.scatter(
        intersection[1], intersection[0], marker="o", ec="black", fc="none", zorder=10
    )
    # mark intersection y positions
    ax.axvline(intersection[1], ls="--", c="forestgreen", alpha=0.5)

ax.text(
    0.2,
    0.9,
    "Stage Movement",
    transform=ax.transAxes,
    ha="center",
    bbox={"facecolor": "white"},
)
ax.text(
    1 - 0.2,
    0.9,
    "Intersection Movement",
    transform=ax.transAxes,
    ha="center",
    bbox={"facecolor": "white"},
);

#### Obtain scaling by linear regression

To obtain a scaling factor we simply plot the stage shift vs the shift of the point of intersection. In the current example we consider a linear relationship. Consequently, linear regression can be utilized. For more complex systems with multiple refractions or curved surfaces this may be not sufficient.

In [ ]:
# convert list of intersections to array
intersection_array = np.array(intersections)

intersection_shifts = intersection_array.T[1] - intersection_array.T[1][0]


fig, ax = plt.subplots()
ax.grid()

ax.scatter(
    np.array(stage_shifts) / step_dy,
    intersection_shifts / step_dy,
    marker="o",
    ec="black",
    fc="none",
    label="simulation",
)
ax.set_xlabel("stage shift")
ax.set_ylabel("intersection shift")


result = linregress(stage_shifts, intersection_shifts)

ax.axline(
    xy1=(0, result.intercept),
    slope=result.slope,
    c="red",
    ls="--",
    label="linear regression",
)

ax.text(
    0.05,
    0.95,
    f"slope: {result.slope:.3f}\nR: {result.rvalue:.3f}",
    transform=ax.transAxes,
    ha="left",
    va="top",
    bbox={"facecolor": "white"},
)

ax.legend();

The obtained slope is **1.343** which matches the value of **1.34** provided by Gunady et al.